<a href="https://colab.research.google.com/github/matheusamorim/spark/blob/main/analise_de_dados_de_psiquiatria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
from pyspark.sql.functions import datediff, to_date, lit
from pyspark.sql import functions as F


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install -q pyspark
!pip install  mmlspark 

import os
import mmlspark
import findspark
import pandas as pd
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

--2020-11-18 12:53:54--  https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 2a01:4f8:10a:201a::2
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 219929956 (210M) [application/x-gzip]
Saving to: ‘spark-3.0.1-bin-hadoop2.7.tgz’

spark-3.0.1-bin-had 100%[===================>] 209.74M  10.9MB/s    in 22s     

2020-11-18 12:54:17 (9.63 MB/s) - ‘spark-3.0.1-bin-hadoop2.7.tgz’ saved [219929956/219929956]

     |████████████████████████████████| 204.2MB 49kB/s 
     |████████████████████████████████| 204kB 31.6MB/s 
     |████████████████████████████████| 4.2MB 4.2MB/s 
  Created wheel for mmlspark: filename=mmlspark-0.0.11111111-cp36-none-any.whl size=4184588 sha256=0ba1c360dc26d0265cb2503be3f0747aff6a72ca29b57a2304ea9c5b356124b2
  Stored in directory: /root/.cache/pip/wheels/84/06/e7/b3fd219ede2a882f

In [3]:
df = spark.read.csv("/content/drive/MyDrive/dados/GO HOSP/2019_HOSP_det.csv", header=True, sep=",")

In [6]:
df2 = spark.read.csv("/content/drive/MyDrive/dados/GO HOSP/2019_HOSP_cons.csv", header=True, sep=',')

In [8]:
df2.show()

+----------+--------+------------------+-----------------+---------------------+---------------+-----------------------+--------------------+------------------+------------+-------------+-------------------+-------------------+---------------+-----------------+-------------------+-----+-----+-----+-----+-----------------------+--------------+------------------------+----------+
| ID_EVENTO|ID_PLANO|IDADE_BENEFICIARIO|SEXO_BENEFICIARIO|CD_MUNIC_BENEFICIARIO|PORTE_OPERADORA|CD_MODALIDADE_OPERADORA|MODALIDADE_OPERADORA|CD_MUNIC_PRESTADOR|UF_PRESTADOR|DT_INTERNACAO|DT_SAIDA_INTERNACAO|CARATER_ATENDIMENTO|TIPO_INTERNACAO|REGIME_INTERNACAO|MOTIVO_ENCERRAMENTO|CID_1|CID_2|CID_3|CID_4|NR_DIARIAS_ACOMPANHANTE|NR_DIARIAS_UTI|LG_VALOR_PREESTABELECIDO|LG_OUTLIER|
+----------+--------+------------------+-----------------+---------------------+---------------+-----------------------+--------------------+------------------+------------+-------------+-------------------+-------------------+-----------

In [9]:
df.createOrReplaceTempView("df")
df2.createOrReplaceTempView("df2")

In [10]:
spark.sql("select * from df2 where TIPO_INTERNACAO = 5.0").show()

+----------+--------+------------------+-----------------+---------------------+---------------+-----------------------+--------------------+------------------+------------+-------------+-------------------+-------------------+---------------+-----------------+-------------------+-----+-----+-----+-----+-----------------------+--------------+------------------------+----------+
| ID_EVENTO|ID_PLANO|IDADE_BENEFICIARIO|SEXO_BENEFICIARIO|CD_MUNIC_BENEFICIARIO|PORTE_OPERADORA|CD_MODALIDADE_OPERADORA|MODALIDADE_OPERADORA|CD_MUNIC_PRESTADOR|UF_PRESTADOR|DT_INTERNACAO|DT_SAIDA_INTERNACAO|CARATER_ATENDIMENTO|TIPO_INTERNACAO|REGIME_INTERNACAO|MOTIVO_ENCERRAMENTO|CID_1|CID_2|CID_3|CID_4|NR_DIARIAS_ACOMPANHANTE|NR_DIARIAS_UTI|LG_VALOR_PREESTABELECIDO|LG_OUTLIER|
+----------+--------+------------------+-----------------+---------------------+---------------+-----------------------+--------------------+------------------+------------+-------------+-------------------+-------------------+-----------

In [37]:
spark.sql("select DT_INTERNACAO, DT_SAIDA_INTERNACAO from df2").show()

+-------------+-------------------+
|DT_INTERNACAO|DT_SAIDA_INTERNACAO|
+-------------+-------------------+
|   01/09/2019|         17/09/2019|
|   01/09/2019|         30/09/2019|
|   01/09/2019|         14/09/2019|
|   01/09/2019|         09/09/2019|
|   01/09/2019|         03/09/2019|
|   01/09/2019|         03/09/2019|
|   01/09/2019|         14/09/2019|
|   01/09/2019|         05/09/2019|
|   01/09/2019|         30/11/2019|
|   01/09/2019|         03/09/2019|
|   01/09/2019|         14/09/2019|
|   01/09/2019|         30/09/2019|
|   01/09/2019|         05/09/2019|
|   01/09/2019|         04/09/2019|
|   01/09/2019|         19/09/2019|
|   01/09/2019|         03/09/2019|
|   01/09/2019|         14/09/2019|
|   01/09/2019|         02/09/2019|
|   01/09/2019|         30/09/2019|
|   01/09/2019|         03/09/2019|
+-------------+-------------------+
only showing top 20 rows



In [47]:
media_internacao = spark.sql('select DT_INTERNACAO, DT_SAIDA_INTERNACAO from  df2')

In [68]:
media_internacao.select(F.mean(datediff(to_date("DT_SAIDA_INTERNACAO", 'dd/MM/yyyy'), to_date("DT_INTERNACAO",'dd/MM/yyyy')))\
                        .alias("Média"))\
                        .show()

+-----------------+
|            Média|
+-----------------+
|3.124821276633071|
+-----------------+

